# **Setup and Library Imports**

### **Installing Required Packages**

In [ ]:
! pip install --quiet "transformers[torch]"
! pip install --quiet evaluate
! pip install --quiet tabulate
! pip install --quiet ipywidgets
! pip install --quiet datasets
! pip install --quiet pillow
! pip install --quiet scikit-learn
! pip install --quiet tensorboard

### **Importing Libraries**

In [ ]:
# PyTorch for tensor operations
import torch

# Hugging Face libraries for training and transformer models
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers import AutoImageProcessor, AutoModelForImageClassification
from transformers import AdamW

# Evaluation metrics and utilities
import evaluate
import numpy as np
from datetime import datetime

# Loading datasets for training and evaluation
from datasets import load_dataset

# Data manipulation and display utilities
import pandas as pd
from tabulate import tabulate
from collections import Counter

### **Logging into Hugging Face Hub**

In [ ]:
# from huggingface_hub import notebook_login
## Execute the login function to access the Hugging Face account
# notebook_login()

### **Connect to google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **Defining Model, Dataset Paths, and Output Directories**

List of Models


```
microsoft/resnet-50
google/vit-base-patch16-224
microsoft/swin-base-patch4-window7-224
facebook/deit-base-patch16-224
microsoft/beit-base-patch16-224
facebook/dinov2-base
```



List of Datasets


```
cvmil/rice-leaf-disease-augmented
cvmil/rice-leaf-disease-augmented-test
cvmil/rice-disease-02
```

Define paths for saving model training outputs and logs, incorporating model and dataset names along with the current date.

In [ ]:
# Define model and dataset paths
model_path = "google/vit-base-patch16-224"
dataset_path = "cvmil/rice-leaf-disease-augmented"
train_epochs = 15

base_model_name = model_path.split("/")[-1]
dataset_name = dataset_path.split("/")[-1]
train_type = "tl" # fft - full fine-tune / tl - transfer learning
dt_string = datetime.now().strftime("%m%d%y")
# dt_string = ""

model_name = f"{base_model_name}_{dataset_name}_{train_type}_{dt_string}"
output_dir = f"./drive/Shareddrives/CS198-Drones/training_output/{model_name}"

# Define directory for storing training logs
logging_dir = f"{output_dir}/logs"

# **Data Preparation and Processing Pipeline**

This section handles the dataset loading, label extraction, image processing setup, and defines necessary functions for data transformation, batching, and metric computation to prepare the data for model training and evaluation.

### **Load Dataset and Extract Labels**

Load the dataset from huggingface and extract the class labels from the training data.

In [ ]:
# Load the dataset
dataset = load_dataset(dataset_path)

# Extract class labels from the training set
labels = dataset['train'].features['label'].names

Generate and display a table showing class distribution across training and validation splits.

In [ ]:
label_mapping = dataset['train'].features['label'].int2str

# Count the number of samples per class in each split
train_counts = Counter(dataset['train']['label'])
validation_counts = Counter(dataset['validation']['label'])

# Create a DataFrame for the class distribution
data = {
    'ID': list(range(len(labels))),
    'Label': labels,
    'Training': [train_counts[i] if i in train_counts else 0 for i in range(len(labels))],
    'Validation': [validation_counts[i] if i in validation_counts else 0 for i in range(len(labels))],
}

# Display the class distribution in a table format
df = pd.DataFrame(data)
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

### **Initialize Image Processor**

Load and initialize the image processor from the pre-trained model.

In [ ]:
# Load the image processor from the pre-trained model
processor = AutoImageProcessor.from_pretrained(model_path)
print(processor)

### **Data Preparation and Processing Pipeline**

Create mappings for label-to-ID and ID-to-label.

In [ ]:
label2id = {c: idx for idx, c in enumerate(labels)}
id2label = {idx: c for idx, c in enumerate(labels)}

Define the transformation function to process the image batch.

In [ ]:
def transforms(batch):
    batch['image'] = [x.convert('RGB') for x in batch['image']]
    inputs = processor(batch['image'], return_tensors='pt')
    inputs['labels'] = batch['label']
    return inputs

Define the custom collation function for batching pixel values and labels.

In [ ]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }


Define the function to compute accuracy during evaluation.

In [ ]:
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


### **Apply Data Transformations to Dataset**

Apply the defined transformation function to the dataset for preprocessing. </br>
Note: This assumes that data augmentation and normalization have already been handled in the previous pipeline and is ready for fine-tuning.

In [ ]:
processed_dataset = dataset.with_transform(transforms)

# **Model Initialization and Trainer Setup**

This section handles the initialization of the model, configuration of training parameters, and setting up the Trainer for fine-tuning, including the datasets, data processing, and evaluation metrics.

### **Initialize Pre-trained Model for Fine-tuning**

Load a pre-trained image classification model, configuring it with the correct label mappings and number of labels for the fine-tuning task.

In [ ]:
# Load pre-trained model and configure it for fine-tuning
model = AutoModelForImageClassification.from_pretrained(
    model_path,                  # Path to the pre-trained model
    num_labels=len(labels),      # Set the number of labels for classification
    id2label=id2label,           # Map from ID to label
    label2id=label2id,           # Map from label to ID
    ignore_mismatched_sizes=True # Ignore size mismatches in weights
)

### **Check Model Parameters for Fine-tuning**

Freeze the model parameters except for those related to the classifier layers, allowing fine-tuning only on the specified layers.

In [ ]:
for name, param in model.named_parameters():
    param.requires_grad = name.startswith(('classifier', 'distillation_classifier'))

We can check how many parameters are there in the model along with how many are actually going to be trained now.

In [ ]:
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {num_params:,} | Trainable parameters: {trainable_params:,}")

### **Define Training Arguments**

Set up the training configuration with parameters such as batch size, number of epochs, learning rate, and logging strategies for the fine-tuning process.

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=64,          # Batch size for training
    per_device_eval_batch_size=64,           # Batch size for evaluation
    num_train_epochs=train_epochs,           # Number of training epochs
    warmup_ratio=0.1,                        # Warmup ratio for learning rate scheduler
    learning_rate=1e-4,                      # Learning rate for optimizer
    weight_decay=0.01,                       # Weight decay for regularization
    save_total_limit=5,                      # Limit the number of saved models
    remove_unused_columns=False,             # Retain unused columns in the dataset
    report_to=['tensorboard'],               # Log to TensorBoard
    save_strategy="epoch",                   # Save strategy
    eval_strategy="epoch",                   # Evaluation strategy
    logging_strategy="epoch",                # Logging strategy
    logging_dir=logging_dir,                 # Directory for logging
    output_dir=output_dir,                   # Directory for saving outputs
    push_to_hub=True,                        # Push model to Hugging Face Hub
    load_best_model_at_end=True,             # Load best model at the end of training
    metric_for_best_model="eval_loss",       # Specify the metric to track
    greater_is_better=False,                 # For loss, lower is better
    resume_from_checkpoint=True,             # Resume from checkpoint
)

### **Initialize Trainer**

Initialize the Trainer object with the model, training arguments, data collator, metrics computation, and datasets for training and evaluation.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    optimizers=(optimizer, None),
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor,
)

### **Trainer Callback**

In [ ]:
class CustomSaveCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        trainer.save_model()
        trainer.save_state()

In [ ]:
trainer.add_callback(CustomSaveCallback)

### **Create Model Card**

In [ ]:
trainer.create_model_card(
    language="en",
    license="MIT",
    tags=["image-classification", "fine-tuning"],
    model_name=model_name,
    finetuned_from=base_model_name,
    tasks=["image-classification"],
    dataset_tags=["image", "rice-leaf_disease"],
    dataset=dataset_name,
    dataset_args=["size: 224x224", "augmentation: true"],
)

# **Model Training and Evaluation**

### **Start Fine-tuning Process**

Initiates the fine-tuning of the model using the Trainer, applying the specified training configurations, such as the batch size, learning rate, and number of epochs. During training, the model will be evaluated at the end of each epoch on the validation dataset using the compute_metrics function, which calculates accuracy.

The model will undergo the following process during fine-tuning:
- **Training**: The model will be trained on the training dataset for the specified number of epochs.
- **Evaluation**: After each epoch, the model will be evaluated on the validation dataset, and accuracy will be computed using the compute_metrics function.
- **Metrics Logging**: The training progress and evaluation results will be logged to TensorBoard and can be monitored during training.

In [ ]:
train_results = trainer.train(resume_from_checkpoint=False)

### **Save Model and Training State**

After the training process, the model and relevant training state are saved. This includes saving the model weights, training metrics, and the state of the trainer, ensuring that training progress can be restored if needed.

In [ ]:
# Save the trained model
trainer.save_model()

# Log and save training metrics for later reference
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

# Save the state of the trainer, including configuration and optimizer state
trainer.save_state()